In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

import json
import time
import rstr
import random

from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

from method.ours.feedback import get_global_feedback
from method.ours.utils import create_driver, get_xpath, interact_with_input

In [3]:
HEADLESS = False
URL = 'https://www.aircanada.com/ca/en/aco/home.html'

In [4]:
driver = create_driver(HEADLESS)
driver.get(URL)

[WDM] - Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 8.29M/8.29M [00:00<00:00, 14.0MB/s]


In [5]:
html = driver.find_element(By.TAG_NAME, 'html').get_attribute('outerHTML')

In [11]:
form = driver.find_elements(By.TAG_NAME, 'form')[1]
form_xpath = get_xpath(driver, form)

inputs = form.find_elements(
    By.XPATH,
    f'{form_xpath}//input | {form_xpath}//textarea | {form_xpath}//select'
)

inputs = list(filter(
    lambda x: x.get_attribute('type') != 'hidden' and x.get_attribute('hidden') != 'true',
    inputs
))

# Generate Base Values

In [12]:
fixed_rules = {
    'text': lambda: 'test',
    'email': lambda: 'test@test.com',
    'password': lambda: 'test',
    'number': lambda: '0',
    'range': lambda: '0',
    'date': lambda: '2020-01-01',
    'time': lambda: '00:00',
    'datetime-local': lambda: '2020-01-01T00:00',
    'week': lambda: '2020-W01',
    'month': lambda: '2020-01',
    'tel': lambda: '0123456789',
    'url': lambda: 'https://test.com',
    'color': lambda: '#000000',
    
    'boolean': lambda: True,
    'select': lambda _: 0,
}


random_rules = {
    'text': lambda: rstr.xeger(r'[A-Za-z0-9]{10}'),
    'email': lambda: rstr.xeger(r'[A-Za-z0-9]{10}') + '@' + rstr.xeger(r'[A-Za-z0-9]{10}') + '.com',
    'password': lambda: rstr.xeger(r'[A-Za-z0-9]{10}'),
    'number': lambda: random.randint(0, 1000),
    'range': lambda: random.randint(0, 1000),
    'date': lambda: rstr.xeger(r'[0-9]{4}') + '-' + rstr.xeger(r'[0-9]{2}') + '-' + rstr.xeger(r'[0-9]{2}'),
    'time': lambda: rstr.xeger(r'[0-9]{2}') + ':' + rstr.xeger(r'[0-9]{2}'),
    'datetime-local': lambda: rstr.xeger(r'[0-9]{4}') + '-' + rstr.xeger(r'[0-9]{2}') + '-' + rstr.xeger(r'[0-9]{2}') + 'T' + rstr.xeger(r'[0-9]{2}') + ':' + rstr.xeger(r'[0-9]{2}'),
    'week': lambda: rstr.xeger(r'[0-9]{4}') + '-W' + rstr.xeger(r'[0-9]{2}'),
    'month': lambda: rstr.xeger(r'[0-9]{4}') + '-' + rstr.xeger(r'[0-9]{2}'),
    'tel': lambda: rstr.xeger(r'[0-9]{10}'),
    'url': lambda: 'https://' + rstr.xeger(r'[A-Za-z0-9]{10}') + '.com',
    'color': lambda: '#' + rstr.xeger(r'[0-9A-Fa-f]{6}'),
    
    'boolean': lambda: True if random.randint(0, 1) == 0 else False,
    'select': lambda element: random.randint(0, len(Select(element).options) - 1),
}

In [13]:
def rule_based_value_generator(inputs, random=True):
    rules = random_rules if random else fixed_rules
    
    values = {}
    
    for element in inputs:
        tag = element.tag_name
        element_xpath = get_xpath(driver, element)
        input_type = element.get_attribute('type') or 'text'
        
        if tag == 'select':
            values[element_xpath] = rules['select'](element)
        
        elif input_type == 'checkbox' or input_type == 'radio':
            values[element_xpath] = rules['boolean']()
        
        else:
            values[element_xpath] = rules[input_type]()
    
    return values

In [14]:
def fill_form_with_values(xpaths, values):
    for element_xpath in xpaths:
        element = driver.find_element(By.XPATH, element_xpath)
        element_value = values[element_xpath]

        if element.get_attribute('type') in ['checkbox', 'radio']:
            continue

        try:
            interact_with_input(element, element_value)
        except Exception as e:
            print(e)
    
    submit_button = driver.find_element(By.XPATH, f'{form_xpath}//button[@type = "submit"]')
    submit_button.click()

In [15]:
base_values = rule_based_value_generator(inputs)

# Fuzz Input Values

In [16]:
fuzz_samples = {}

with open('fuzz-samples.json', 'r') as f:
    fuzz_samples = json.load(f)

In [17]:
tested = []


for element_xpath in base_values.keys():
    new_values = dict(base_values)
    element = driver.find_element(By.XPATH, element_xpath)
    
    print(element.get_attribute('outerHTML'))
    print()
    
    if element.get_attribute('type') not in fuzz_samples:
        time.sleep(1)
        continue
    
    fuzz_sample = fuzz_samples[element.get_attribute('type')]
    
    for i, (sample_name, sample_values) in enumerate(fuzz_sample.items()):
        if i == 1:
            break
        for sample_value in sample_values:
            driver.get(URL)
            time.sleep(3)
            html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
            
            new_values[element_xpath] = sample_value
            fill_form_with_values(base_values.keys(), new_values)
            diff = get_global_feedback(
                html,
                driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
            )
            tested.append({
                "fuzz_type": sample_name,
                "values": new_values,
                "changes": '-'.join(diff),
            })

<input type="radio" class="abc-form-element-main abc-form-element-radio-button ng-untouched ng-pristine ng-valid" id="bkmgFlights_tripTypeSelector_R" name="tripType" value="R" aria-disabled="false" aria-invalid="false" aria-checked="true">

<input type="radio" class="abc-form-element-main abc-form-element-radio-button ng-untouched ng-pristine ng-valid" id="bkmgFlights_tripTypeSelector_O" name="tripType" value="O" aria-disabled="false" aria-invalid="false" aria-checked="false">

<input type="radio" class="abc-form-element-main abc-form-element-radio-button ng-untouched ng-pristine ng-valid" id="bkmgFlights_tripTypeSelector_M" name="tripType" value="M" aria-disabled="false" aria-invalid="false" aria-checked="false">

<input type="checkbox" role="checkbox" class="abc-form-element-main abc-form-element-checkbox ng-untouched ng-pristine ng-valid" id="bkmgFlights_searchTypeToggle" aria-label="Book with Aeroplan points" aria-disabled="false" aria-required="false" aria-invalid="false" aria-che

In [18]:
df = pd.DataFrame(tested)

In [19]:
errors = df.groupby('changes').first()

In [20]:
errors.to_csv('./random.csv')